In [ ]:
====가변의 길이 인수(순전파 편)=====
가변 길이: 인수 또는 반환값의 수가 달라질 수 있다.

    
가변 길이 입출력 표현: '하나의 인수'만 받고 '하나의 값'만 반환

In [2]:
import unittest
import numpy as np

In [9]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output #함수의 입출력을 얻는다.
            x.grad = f.backward(y.grad) #backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [11]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
====가변의 길이 인수(개선 편)=====
#개선사항: 리스트나 튜플을 거치지않고 인수와 결과를 직접 주고 받는다.
#1. Add클래스(혹은 다른 구체적인 함수 클래스)를 '사용하는 사람'을 위한 개선
#2. '구현하는 사람'을 위한 개선

In [ ]:
===1. Add클래스(혹은 다른 구체적인 함수 클래스)를 '사용하는 사람'을 위한 개선==

In [4]:
class Function:
    def __call__(self, *inputs): #별표를 붙여야한다.>가변인수길이를 건네 함수 호출
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(*xs)#별표를 붙여 언팩
        if not isinstance(ys, tuple): #튜플이 아닌 경우 추가 지원
            ys = (ys, )
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] #리스트의 원소가 하나라면 첫 번째 원소를 반환한다.

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [22]:
# class Add(Function):
#     def forward(self, xs):
#         x0, x1 = xs
#         y = x0 + x1
#         return (y,)


In [5]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [7]:
def add(x0, x1):
    return Add()(x0, x1)

In [12]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [23]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [5]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [6]:
def square(x):
    return Square()(x)

In [7]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [8]:
class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [ ]:
==13==

In [ ]:
#덧셈의 역전파는 출력 쪽에서 전해지는 미분값에 1을 곱한 값이 입력 변수(x0, x1)의 미분
##즉 상류에서 흘러오는 미분밗을 '그대로 흘려보내는 것'is 덧셈의 역전파

In [13]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] #출력 변수인 outputs에 담긴 미분값을 리스트로 담는다.
            gxs = f.backward(*gys) #함수 f의 역전파 호출
            if not isinstance(gxs, tuple): #gxs가 튜플이 아니면 튜플로 변환
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs) #역전파로 전파되는 Variable의 인스턴스 변수 grad에 저장
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)


In [14]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [15]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [16]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [17]:
def square(x):
    f = Square()
    return f(x)

In [18]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [19]:
def add(x0, x1):
    return Add()(x0, x1)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

##Dezero를 사용하여 z=x^2+y^2이라는 계싼을 z=add(square(x),square(y))라는 코드로 해결

13.0
4.0
6.0


In [ ]:
==14==

In [ ]:
#같은 변수를 반복해서 사용할 경우 의도대로 동작하지 않을 수 있다. 
##>>해결책은 미분값(grad)을 처음 설정하는 경우에는 지금까지와 똑같이 출력 쪽에서 전해진느 미분값을 그대로 대입!
##>>>다음부터는 전달된 미분값을 '더해'주도록 수정.

In [21]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() #처리할 함수를 그 리스트의 끝에서 꺼냄
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator) #함수의 후보를 funcs 리스트의 끝에 추가

In [22]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [23]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [24]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


In [25]:
def add(x0, x1):
    return Add()(x0, x1)

In [26]:
#첫번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


In [27]:
#두번째 계싼(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad() #미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad)

3.0


In [ ]:
==15==

In [29]:
#지금까지는 일직선 계산 그래프의 미분 계산만 다뤘음
#복잡하게 연결된 계산 그래프의 미분 계산 필요.
>> 변수를 반복해서 사용하면 역전파 때는 출력쪽에서 전파하는 미분값을 더해라.

In [30]:
#Dezero의 문제점
##함수 순서 처리가 D,C,A,B,A가 된다. C 다음에 A가 바로 이어짐
##함수 A의 역전파가 두번 일어나는 것도 문제

In [ ]:
#문제 원인
##한줄로 나열된 계산 그래프를 다룸.
##리스트에서 함수를 꺼내 처리하는 순서 고려안함.
##Funcs 리스트 [B,A]에서 마지막 원소 A가 꺼내진다.

In [ ]:
#함수 우선순위
##funcs리스트에서지금까지 '마지막 원소'만 꺼냄
##순전파 때 '함수'가 '변수'를 만들어냄. 이는 '부모-자식 관계'임
##이 관계를 기준으로 함수와 변수의 세대를 기록
##세대가 우선순위에 해당
##역전파 시 세대수가 큰 쪽부터 처리하면 '부모'보다 '자식'이 먼저 처리됨

In [ ]:
==16==

In [ ]:
#복잡한 계산 그래프
##함수 우선순위는 순전파 시 '세대'를 설정(역전파시에는 최근 세대의 함수부터)
#순전파에서 세대추가하고 역전파에서는 세대 순으로 꺼냄

In [39]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f): #함수 리스트를 세대 순으로 정렬하는 역할을 수행
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

In [40]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0
